$Final \ \ Code$

In [ ]:
# @title $Start$
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt
from scipy import linalg
from functools import reduce

def modulo_n(n,f):
    f = sp.Poly(f)
    Gen = f.gens
    Coef = f.coeffs()
    Mono = f.monoms()
    for i in range(len(Coef)):
        Coef[i] = Coef[i]%n
    expr=0
    for i in range(len(Coef)):
        monomial=1
        for j in range(len(Gen)):
            monomial=monomial*Gen[j]**Mono[i][j]
        expr=expr+Coef[i]*monomial
    return(expr)

In [ ]:
# @title $Enter \ \ Generators \ \ of \ \ Ideal$

import sympy as sp
import numpy as np
import itertools  #cross products of lists


d0=int(input('Number of Generators : ')) # d0 - number of generators
generators=[]                            # Generators of an Ideal

for i in range(d0):
    print('enter g',i+1,':',sep='')
    gen_temp=input(": ")
    gen_temp=sp.sympify(gen_temp)
    generators.append(gen_temp)

In [ ]:
#@title $Choose \ \ Base \ \ Ring$
choose_ring = "Z(n)" #@param ["C","R", "Q", "Z", "Z(n)"] {allow-input: true}

if (choose_ring == "C"):
    ring="CC"
elif (choose_ring == "R"):
    ring="RR"
elif (choose_ring == "Q"):
    ring="QQ"
elif (choose_ring == "Z"):
    ring="ZZ"
elif (choose_ring == "Z(n)"):
    d0=int(input('n = '))
    ring="GF("+str(d0)+")"
    #ring="ZZ("+str(d0)+")"

In [ ]:
#@title $Variables \ \ and \ \ Order$
choose_order = "lex" #@param ["lex", "grlex", "grevlex"]
X = [sp.sympify(i) for i in input("Enter variables separated by ',':").split(',')]
v = dict([(i.name, i) for i in X])   #v - variables


In [ ]:
#@title $Grobner \ \ Basis \ \ for \ \ Ideal$ { form-width: "20%" }
algorithm = "buchberger" #@param ["buchberger", "f5b"]
from IPython.display import HTML, Math
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
    "latest.js?config=default'></script>"))

display(Math(r'Grobner \ \ Basis \ \ G \ ='))
var=[] # list of variables
for i in range(len(X)):
    var.append(X[i])

G=sp.groebner(generators, var, domain=ring, order=choose_order, method=algorithm)     # Grobner Basis
for i in range(len(G)):
    display(Math(r'{\large g_{_{'+sp.latex(i+1)+r'}}='+sp.latex(sp.nsimplify(G[i]))+r'}'))
    print()
try:
    check_zero_dim = sp.is_zero_dimensional(G,order=choose_order)
    if (check_zero_dim==1):
        display(Math(r'Ideal \ \ is \ \ Zero \ \ Dimensional \ !'))
    else:
        display(Math(r'Ideal \ \ is \ \ Not \ \ a \ \ Zero \ \ Dimensional \ !!'))
except:
    pass

if(check_zero_dim==1):
    print()
    print()
    display(Math(r'Leading \ \  Monomials \ \ of \ \ G '))
    LM_G=[]    # Leading Monomials of Grobner Basis
    for i in range(len(G)):
        temp=(sp.LM(G[i],var,order=choose_order))
        LM_G.append(temp)
        display(Math(r'{\large LM \left( g_{_{'+sp.latex(i+1)+r'}} \right)='+sp.latex(temp)+r'}'))
        print()

    expo_vect_LT=[]   # Exponent of LT/LM (exponent_x, exponent_y)
    for i in range(len(LM_G)):
        expo_vect_LT.append(sp.degree_list(LM_G[i],tuple(X)))

    max_exp=[] # Maximum of exponents
    expo_vect1=np.array(expo_vect_LT) # Exponent of LT/LM (exponent_x, exponent_y)
    for i in range(len(X)):
        max_exp.append(max(expo_vect1[:,i]))

    total_monomials=1
    for i in range(len(max_exp)):
        total_monomials=total_monomials*max_exp[i]

    list0=[] # exponents of x,y cordinates
    for i in range(len(max_exp)):
        temp_list=[(j) for j in range(max_exp[i])]
        list0.append(temp_list)

    list0
    expo_vect=[] # exponents of std basis
    for element in itertools.product(*list0):
        expo_vect.append(element)

    k=0
    while (k < len(expo_vect_LT)):
        vect_lt=expo_vect_LT[k]
        j=0
        while (j < len(expo_vect)):
            vect=expo_vect[j]
            check=0
            for i in range(len(vect)):
                if (vect_lt[i] <= vect[i] and check==0):
                    pass
                else:
                    check=1
                    break

            if (check==0):
                expo_vect.remove(vect)
            else:
                j=j+1

        k=k+1

    print()
    display(Math(r'\therefore \ \ Standard \ \ Basis  \ \ is:'))
    print()
    std_basis=[]
    x=sp.symbols('x')
    y=sp.symbols('y')
    z=sp.symbols('z')
    for i in range(len(expo_vect)):
        poly=1
        for j in range(len(expo_vect[i])):
            poly=poly*var[j]**expo_vect[i][j]
        std_basis.append(poly)

    expr=r' {\large \ B= \left \{ '
    for i in range(len(std_basis)):
        expr = expr + sp.latex(std_basis[i])
        if (i != (len(std_basis)-1)):
            expr = expr+r' \ , \ '
    expr=expr+ r' \right \} } '
    display(Math(expr))
    print()
    display(Math(r'Dimension ='+sp.latex(len(std_basis))))


In [ ]:
#@title $Quotient \ \ Ring$

temp=r'\left \langle '
for i in range(len(generators)):
    temp=temp+sp.latex(generators[i])
    if (i != (len(generators)-1)):
        temp=temp + r' \ , \ '
temp=temp+r'\right \rangle '
if(choose_ring == "Z(n)"):
    quotient_ring=r'\frac{ Z_{'+sp.latex(d0)+r'}'+sp.latex(var)+r'} {'+temp+r'}'
else:
    quotient_ring=r'\frac{'+sp.latex(choose_ring)+sp.latex(var)+r'} {'+temp+r'}'

temp1=r' \left \langle '
for j in range(len(std_basis)):
    temp1=temp1 + r'\bar{' + sp.latex(std_basis[j]) + r'}'
    if (j != (len(std_basis)-1)):
        temp1=temp1 + r' \ , \ '
temp1=temp1 + r' \right \rangle }'

display(Math(r'{\large '+quotient_ring + r'=' +temp1))
print()
print()
k = len(std_basis)
n=d0**k
coeff=[[(0) for j in range(k)]]
for i in range(1,n):
    list0=[(0) for j in range(k)]
    count=0
    while (i > 0):
        r = i % d0
        list0[count]=r
        i = i // d0
        count=count+1
    coeff.append(list0)
QRing=[] # Quotient ring
for every in coeff:
    elt=0
    QRing.append(sum([(every[i]*std_basis[i]) for i in range(k)]))

display(Math(r'Total \ \ Number \ \ of \ \ Elements \ ='+sp.latex(len(QRing))))
# for elt in QRing:
#     display(Math(sp.latex(elt)))


In [ ]:
# @title $Non-Units \ \ of \ \ Ring \ \ R \ : \ R^{*}$
non_units = [] # Dictionary of zero divisors of quotient ring
red_elts={} # DICTIONARY OF ELEMENT AND ITS REDUCED form
constants = {}
modulo = {}

count= 0
for f in QRing:
    count=count+1
    print(count,'/',len(QRing),sep="",end=',')
    if(count%25==0):
        print()
    for g in std_basis:
        h0 = sp.expand(f * g)
        if(h0 in red_elts.keys()):
            h=red_elts[h0]
        else:
            h1 = sp.reduced(h0, G, var, domain=ring, order=choose_order)[1]
            if(h1 in constants.keys()):
                h = constants[h1]
            else:
                if(h1.is_constant()):
                    #print(h1)
                    h=h1%d0
                    constants[h1] = h
                    #print(h1)
                else:
                    if(h1 in modulo.keys()):
                        h = modulo[h1]
                    else:
                        h=modulo_n(d0,h1)
                        modulo[h1] = h


            red_elts[h0]=h

        if(h == 0):
            non_units.append(f)
            break

units = [elt for elt in QRing if elt not in non_units]

In [ ]:
#@title $Associate \ \ Classes \ \ of \ \ R$ { form-width: "20%" }
#@markdown $ [a]=[b]$ $ \space iff \space$ $ a $ $=$ $ub$

units = [elt for elt in QRing if elt not in non_units] # units of ring

associates={}
count=0
for elt1 in non_units:
    count=count+1
    print(count,'/',len(non_units),sep="",end=',')
    if(count%25==0):
        print()
    associates[elt1]=set()
    for elt2 in units:
        h0 = sp.expand(elt1 * elt2)
        if(h0 in red_elts.keys()):
            h=red_elts[h0]
        else:
            h1 = sp.reduced(h0, G, var, domain=ring, order=choose_order)[1]
            if(h1 in constants.keys()):
                h = constants[h1]
            else:
                #if(h1.is_constant()):
                if(str(type(h1)).split('.')[:3]==["<class 'sympy", 'core', 'numbers']):
                    h=h1%d0
                    constants[h1] = h
                else:
                    h=modulo_n(d0,h1)
                    modulo[h1] = h

        red_elts[h0]=h
        associates[elt1].add(h)
print()
for elt in associates:
    associates[elt]=list(associates[elt])

temp=[]
for elt in associates:
    if (associates[elt]==list([0])):
        temp.append(elt)

for elt in temp:
    del associates[elt]

print()
zn_star = [] # vertices for compressed asso class graph
for elt in associates:
    zn_star.append(elt)

eq_class=[]
i=0
while (len(zn_star) >= 1):
    elt1=zn_star[i]
    temp=[elt1]
    zn_star.remove(elt1)
    for elt2 in associates:
        if (elt1 != elt2 and set(associates[elt1])==set(associates[elt2])):
            temp.append(elt2)
            zn_star.remove(elt2)

    eq_class.append(temp)

repAsso_class={} # representative of EQ class and corresponding EQ class

for i in range(len(eq_class)):
    cls=eq_class[i]
    repAsso_class[cls[0]]=cls



In [ ]:
# @title $Generators \ \ of \ \ Ideals$

idl_gens = repAsso_class.keys()
display(Math(sp.latex(list(idl_gens))))
print()
display(Math(r'No. \ \ of \ \ Generators \ :' + sp.latex(len(idl_gens))))


In [ ]:
# @title $Principal \ \ Ideals$

Pr_ideals = {}  #Principal Ideals #Dictionary of all R multiples of representatives(Generators) of orbits
All_ideals = {}
for g in idl_gens:
    temp = []
    for e in QRing:
        h0 = sp.expand(g*e)
        if(h0 in red_elts.keys()):
            h = red_elts[h0]
        else:
            h = sp.reduced(h0, G, var, domain=ring, order=choose_order)[1]
            if(h==0):
                red_elts[h0]=h
            else:
                h = modulo_n(d0,h)
                red_elts[h0]=h

        if(h not in temp):
            temp.append(h)

    check = 1
    for ideal0 in All_ideals.values():
        if(set(temp) == set(ideal0)):
            check = 0
            break
    if(check==1):
        Pr_ideals[g] = temp
        All_ideals[tuple((g,))] = set(temp)

display(Math(r'All \ \ Principal \ \ Ideals:'))
for key in Pr_ideals.keys():
    display(Math(r'{Generator \ : \ ' + sp.latex(key)+r'}'))
    #display(Math(r'{Ideal \ : \ ' + sp.latex(Pr_ideals[key])+r'}'))
    display(Math(r'Size ='+sp.latex(len(Pr_ideals[key]))))
    print('------------------------------------------------------')

print()
display(Math(r'No. \ \ of \ \ Principal \ \ Ideals \ :'+sp.latex(len(Pr_ideals.keys()))))

# $Doubly \ \ Generated \ \ Idels$

In [ ]:
#@title $All \ \ two \ \ combinations \ \ of \ \ generators$

#Remove principal ideals who has cardinality = |m| m-maximal ideal

pr_idl_gens = list(idl_gens)
N = len(non_units)
if(N%2==0):
    len_idl = N//2
else:
    i = int(np.ceil(N/2))
    while(N%i != 0):
        i = i-1
    len_idl = i

for key in Pr_ideals.keys():
    if(len(Pr_ideals[key]) == len_idl):
        pr_idl_gens.remove(key)

len_gens = len(pr_idl_gens) #No. of associate classes
two_comb=[] # All combinations of representatives of orbits

two_comb.extend(itertools.combinations(pr_idl_gens, 2))

display(Math(r'No. \ \ of \ \ all \ \ possible \ two \ combinations \ : \ ' + sp.latex(len(two_comb))))


In [ ]:
# @title $Ideals \ \ generated \ \ by \ \ two \ \ elements$
#all_comb
gen_max_idl = [set(X)]
k_gen_ideals = {}  # Dictionary of all Ideals generated by two elements
if(True):
    for gens in two_comb: #gens -> generators
        temp_list = [Pr_ideals[elt] for elt in gens] # list of list of principal ideals generated by generators in gens
        all_two_comb = list(set(list(itertools.product(*temp_list, repeat=1))))
        temp=[]
        check = 1
        for tup0 in all_two_comb:
            if(set(tup0) in gen_max_idl):
                check=0
            else:
                if(len(temp) > len_idl or set(X).issubset(set(temp))):
                    gen_max_idl.append(set(gens))
                    check = 0
                    break
                else:
                    h0 = sp.expand(sum(tup0))
                    if(h0 in red_elts.keys()):
                        h = red_elts[h0]
                    else:
                        h = sp.reduced(h0, G, var, domain=ring, order=choose_order)[1]
                        if(h==0):
                            red_elts[h0]=h
                        else:
                            h = modulo_n(d0,h)
                            red_elts[h0]=h

                    if(h not in temp):
                        temp.append(h)

        if(check == 1):
            for ideal0 in All_ideals.values():
                if(set(temp) == set(ideal0)):
                    check = 0
                    break

        if(check==1):
            k_gen_ideals[gens]=set(temp)
            All_ideals[gens]=set(temp)

display(Math(r'All \ \ two \ \ Generated \ \ Ideals:'))
for key in k_gen_ideals.keys():
    display(Math(r'{Generator \ : \ ' + sp.latex(key)+r'}'))
    #display(Math(r'{Ideal \ : \ ' + sp.latex(k_gen_ideals[key])+r'}'))
    display(r'Size ='+sp.latex(len(k_gen_ideals[key])))
    print('------------------------------------------------------')

print()
display(Math(r'No. \ \ of \ \ Doubly \ \ Generated \ \ Ideals \ :'+sp.latex(len(k_gen_ideals.keys()))))

In [ ]:
# @title $All \ \ Ideals$
All_ideals[tuple(X)] = set(non_units)

# Remove k_gen_ideals whose cardinality is =|m|/2
if(N%2==0):
    len_idl = N//2
else:
    i = int(np.ceil(N/2))
    while(N%i != 0):
        i = i-1
    len_idl = i

stop = 0
while(stop==0):
    k_idl_gens = [] #generators of k-generated ideals
    for key0 in k_gen_ideals.keys():
        k_idl_gens.append(key0)

    # Remove k_gen_ideals whose cardinality is =|m|/2
    for key in k_gen_ideals.keys():
        if(len(k_gen_ideals[key]) >= len_idl):
            k_idl_gens.remove(key)

    kp1_gen_ideals = {} #ideals generated by k+1 generators
    for key1 in k_idl_gens:
        for key2 in pr_idl_gens:
            if(key2 not in key1):
                if(any((tup0) in (set(key1).union({key2})) for tup0 in gen_max_idl)):
                   break
                else:
                #if(True):
                    temp_ideal = set()
                    for elt1 in k_gen_ideals[key1]:
                        for elt2 in Pr_ideals[key2]:
                            h0 = sp.expand(sum([elt1,elt2]))
                            if(h0 in red_elts.keys()):
                                h = red_elts[h0]
                            else:
                                h = sp.reduced(h0, G, var, domain=ring, order=choose_order)[1]
                                if(h==0):
                                    red_elts[h0]=h
                                else:
                                    h = modulo_n(d0,h)
                                    red_elts[h0]=h

                            temp_ideal.add(h)

                    check=1
                    for ideal0 in All_ideals.values():
                        if(temp_ideal == ideal0):
                            check = 0
                            break

                    if(check==0):
                        pass
                    else:
                        gens = tuple(set(key1).union({key2}))
                        k_gen_ideals[gens]=temp_ideal
                        All_ideals[gens]=temp_ideal
                        kp1_gen_ideals[gens]=temp_ideal

    if(len(kp1_gen_ideals.keys())==0):
        stop=1
    else:
        k_gen_ideals = {}
        for key3 in kp1_gen_ideals.keys():
            k_gen_ideals[key3] = kp1_gen_ideals[key3]

for i in range(1,len(idl_gens)):
    display(Math(r'No. \ \ of \ \ Generators =' + sp.latex(i)))
    for key in All_ideals.keys():
        try:
            if(len(key)==i):
                #display(Math(r'{' + sp.latex(key) + r'_{'+ sp.latex(i) + r'} \ : \ ' + sp.latex(All_ideals[key]) +r'}'))
                #display(Math(r'{' + sp.latex(key) + r'_{'+ sp.latex(i) + r'} \ : \ ' + sp.latex(All_ideals[key]) +r'}'))
                #display(Math(r'{ |' + sp.latex(key) +r'| = ' + sp.latex(len(All_ideals[key])) +r'}'))
                display(Math(r'{Generator \ : \ ' + sp.latex(key)+r'}'))
                #display(Math(r'{Ideal \ : \ ' + sp.latex(All_ideals[key])+r'}'))
                display(r'Size ='+sp.latex(len(All_ideals[key])))
                print()
        except:
            pass


In [ ]:
# @title $a,b \in I \implies a-b \in I$
count =1
for key in All_ideals:
    print('Ideal no:',count,sep='', end=' , ')
    if(count%10==0):
        print()
    Ideal = All_ideals[key]
    for elt1 in Ideal:
        for elt2 in Ideal:
            h0 = sp.expand(elt1-elt2)
            if(h0 in red_elts.keys()):
                h = red_elts[h0]
            else:
                h = sp.reduced(h0, G, var, domain=ring, order=choose_order)[1]
                if(h==0):
                    red_elts[h0]=h
                else:
                    h = modulo_n(d0,h)
                    red_elts[h0]=h

            if(h not in Ideal):
                print(elt1)
                print(elt2)
                print(Ideal)
                print('------')
    count = count + 1

In [ ]:
# @title $a \in R, \ b \in I \implies ab \in I$
count =1
for key in All_ideals:
    print('Ideal no:',count,sep='', end=' , ')
    if(count%10==0):
        print()
    Ideal = All_ideals[key]
    for elt1 in Ideal:
        for elt2 in QRing:
            h0 = sp.expand(elt1*elt2)
            if(h0 in red_elts.keys()):
                h = red_elts[h0]
            else:
                h = sp.reduced(h0, G, var, domain=ring, order=choose_order)[1]
                if(h==0):
                    red_elts[h0]=h
                else:
                    h = modulo_n(d0,h)
                    red_elts[h0]=h

            if(h not in Ideal):
                print(elt1)
                print(elt2)
                print(Ideal)
                print('------')

    count = count + 1


In [ ]:
# @title $All \ \ Ideals$
for key in All_ideals.keys():
    display(Math(r'{' + sp.latex(key) + r' \ : \ ' + sp.latex(len(All_ideals[key])) +r'}'))
    print('------------')
print('Total Ideals :', len(All_ideals.keys()))

In [ ]:
# @title $UC \  and  \ LC$
LC_Poset = {}
UC_Poset = {}
LC0_Poset = {}
UC0_Poset = {}
Poset0 = []
for key1 in All_ideals.keys():
    temp_lc=[]
    temp_uc=[]
    for key2 in All_ideals.keys():
        if((key1 != key2) and (All_ideals[key2].issubset(All_ideals[key1]))):
            temp_lc.append(key2)
    for key2 in All_ideals.keys():
        if((key1 != key2) and (All_ideals[key1].issubset(All_ideals[key2]))):
            temp_uc.append(key2)

    LC_Poset[key1] = temp_lc
    UC_Poset[key1] = temp_uc
    LC0_Poset[key1] = temp_lc
    UC0_Poset[key1] = temp_uc
    Poset0.append(key1)
#display(Math(sp.latex(LC_Poset)))
print()
#display(Math(sp.latex(UC_Poset)))

In [ ]:
# @title $Levels$

idls_cards = set()#cardinalities of ideals
for key in All_ideals.keys():
    idls_cards.add(len(All_ideals[key]))
idls_cards = list(np.sort(list(idls_cards))[::-1])
#idls_cards

Poset_levels = {}
for i in range(len(idls_cards)):
    Poset_levels[i] = []
#Poset_levels

# Find 1st level
for key0 in Poset0:
    if(len(All_ideals[key0])==N):
        Poset_levels[0].append(key0)
        break
Poset0.remove(key0)
#Poset_levels
idls_cards.remove(idls_cards[0])


# Find Higher Levels
i=1
while(len(idls_cards) != 0):
    to_delete = []
    for key0 in Poset0:
        if(len(All_ideals[key0]) == idls_cards[0]):
            Poset_levels[i].append(key0)
            to_delete.append(key0)

    for key0 in to_delete:
        Poset0.remove(key0)

    j=1
    while(j < len(idls_cards)):
        to_delete = []
        for key0 in Poset0:
            if(len(All_ideals[key0]) == idls_cards[j]):
                if(len(set(UC_Poset[key0]).intersection(set(Poset_levels[i])))==0):
                    Poset_levels[i].append(key0)
                    to_delete.append(key0)

        for key0 in to_delete:
            Poset0.remove(key0)
        j = j+1

    idls_cards.remove(idls_cards[0])
    i=i+1

for i in range(len(Poset_levels.keys())):
    display(Math(r'Level :'+sp.latex(i)))
    for key in Poset_levels[i]:
        display(Math(sp.latex(key)+r';'+sp.latex(len(All_ideals[key]))))
    print('------')

In [ ]:
# @title $Poset$



for key1 in All_ideals.keys():
    for i in range (len(Poset_levels)):
        for key2 in Poset_levels[i]:
            if(key2 in LC_Poset[key1]):
                LC0_Poset[key1] = [key3 for key3 in LC0_Poset[key1] if key3 not in LC_Poset[key2]]

for key1 in All_ideals.keys():
    for i in range (len(Poset_levels)):
        for key2 in Poset_levels[i]:
            if(key2 in UC_Poset[key1]):
                UC0_Poset[key1] = [key3 for key3 in UC0_Poset[key1] if key3 not in UC_Poset[key2]]


import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 8), dpi=120, facecolor='w', edgecolor='k')

plt.scatter(0,0, color='red') #Plot maximal ideal
plt.text(0,0,'$m$')
ideal_cords = {}
ideal_cords[Poset_levels[0][0]]=(0,0)
for i in range(1,len(Poset_levels)):
    n = len(Poset_levels[i])
    y_cords = [(-1*i) for j in range(n)]
    x_cords = list(range(-1*(n-1),n,2))
    plt.scatter(x_cords,y_cords, color='red') #Plot maximal ideal


    count=0
    for key0 in Poset_levels[i]:
        ideal_cords[key0] = (x_cords[count], y_cords[count])
        plt.text(x_cords[count],y_cords[count],r'$'+sp.latex(Poset_levels[i][count])+r'$',rotation = 45, fontsize=10)
        count = count + 1

# for i in range(len(Poset_levels)-1):
#     for key1 in Poset_levels[i]:
#         for key2 in Poset_levels[i+1]:
#             if(key1 in LC_Poset[key2]):
#                 plt.plot([ideal_cords[key1][0],ideal_cords[key2][0]],[ideal_cords[key1][1],ideal_cords[key2][1]], color='black',linewidth=1)

col=['red','yellow', 'green','pink','blue','violet','purple']

for key1 in All_ideals.keys():
    #count = 0
    for key2 in UC0_Poset[key1]:
        plt.plot([ideal_cords[key1][0],ideal_cords[key2][0]],[ideal_cords[key1][1],ideal_cords[key2][1]],linewidth=1)

plt.axis('off')
plt.show()